# Design and implement an A* Search algorithm using Python for solving a 2x2 Rubik’s Cube.

Integrantes:
- Dylan Samuel Cantillo Arrieta
- Laura Mariana González Solano
- Santiago Pelmet

### Library

In [1]:
from scipy.sparse import coo_matrix   # Initial matrix with empty spaces
import numpy as np                    # Operationes with arrays
import matplotlib.pyplot as plt       # 2d graph
import plotly.graph_objects as go     # 3d graph
import copy, os                       # Copy elements, files

# Colors in console
color_amarillo = '\033[1;33;1m'
color_azul = '\033[1;34;1m'
color_blanco = '\033[0;0;0m'

### Cube initial values

In [2]:
# Positiones left to right and up to down in a matrix
# The integer part define the color and the float part define the piece of color
data = [1.1, 1.2, 1.3, 1.4, 2.1, 2.2, 3.1, 3.2, 4.1, 4.2, 5.1, 5.2, 2.3, 2.4, 3.3, 3.4, 4.3, 4.4, 5.3, 5.4, 6.1, 6.2, 6.3, 6.4]
I = [0]*2 + [1]*2 + [2]*8 + [3]*8 + [4]*2 + [5]*2
J = [2, 3]*2 + [*range(8)]*2 + [2, 3]*2
matrix_cube = coo_matrix((data, (I, J))).toarray()

In [3]:
print(matrix_cube)

[[0.  0.  1.1 1.2 0.  0.  0.  0. ]
 [0.  0.  1.3 1.4 0.  0.  0.  0. ]
 [2.1 2.2 3.1 3.2 4.1 4.2 5.1 5.2]
 [2.3 2.4 3.3 3.4 4.3 4.4 5.3 5.4]
 [0.  0.  6.1 6.2 0.  0.  0.  0. ]
 [0.  0.  6.3 6.4 0.  0.  0.  0. ]]


Rubik's cube 2x2 movements

In [4]:
clockwise_movements = ['F', 'R', 'B', 'L', 'U', 'D']
counterclockwise_movements = ['F\'', 'R\'', 'B\'', 'L\'', 'U\'', 'D\'']

movements = clockwise_movements + counterclockwise_movements

In [5]:
names_movements = {
    'F': 'orange-right',
    'R': 'blue-right',
    'B': 'red-right',
    'L': 'green-right',
    'U': 'yellow-right',
    'D': 'white-right',
    'F\'': 'orange-left',
    'R\'': 'blue-left',
    'B\'': 'red-left',
    'L\'': 'green-left',
    'U\'': 'yellow-left',
    'D\'': 'white-left'
}

### 3D graph

In [6]:
def get_color_for_cube3D(matrix):
    """
    Set colors of each triangle of 3d graph consider the integer of the matrix
    """
    dicc = {
        1: 'yellow',
        2: 'green',
        3: 'orange',
        4: 'blue',
        5: 'red',
        6: 'white'
    }
    colors = []
    for i in range(2, 8):
        colors += [dicc[int(matrix[2, i])]]*2
    colors += [dicc[int(matrix[2, 0])]]*2 +  [dicc[int(matrix[2, 1])]]*2
    for i in range(2, 8):
        colors += [dicc[int(matrix[3, i])]]*2
    colors += [dicc[int(matrix[3, 0])]]*2 + [dicc[int(matrix[3, 1])]]*2

    colors += [dicc[int(matrix[1, 2])]]*2 + [dicc[int(matrix[1, 3])]]*2 + [dicc[int(matrix[0, 3])]]*2 + [dicc[int(matrix[0, 2])]]*2 
    colors += [dicc[int(matrix[4, 2])]]*2 + [dicc[int(matrix[4, 3])]]*2 + [dicc[int(matrix[5, 3])]]*2 + [dicc[int(matrix[5, 2])]]*2 
    return colors

In [7]:
def show_cube(matrix, duration):
    """
    Fill faces and create lines for show the cube considering a matrix
    """
    colors = get_color_for_cube3D(matrix)

    sup = [[0, 0, 2], [1, 0, 2], [2, 0, 2], [2, 1, 2], [2, 2, 2], [1, 2, 2], [0, 2, 2], [0, 1, 2]]
    med = [[0, 0, 1], [1, 0, 1], [2, 0, 1], [2, 1, 1], [2, 2, 1], [1, 2, 1], [0, 2, 1], [0, 1, 1]]
    inf = [[0, 0, 0], [1, 0, 0], [2, 0, 0], [2, 1, 0], [2, 2, 0], [1, 2, 0], [0, 2, 0], [0, 1, 0]]

    cubo = sup + med + inf + [[1, 1, 2], [1, 1, 0]]
    X, Y, Z = [f[0] for f in cubo], [f[1] for f in cubo], [f[2] for f in cubo]

    I, J, K = [], [], []
    for i in range(8):
        I += [i%8, (i+1)%8]
        J += [(i+1)%8, (i+8)%16]
        K += [(i+8)%16, (i+9)%16 + 8 if (i+9)%16 < 8 else (i+9)%16]

    I, J, K = I+[*np.array(I)+8], J+[*np.array(J)+8], K+[*np.array(K)+8]

    I, J, K = I+[7], J+[0], K+[len(cubo)-2]
    for i in range(7):
        I += [i]
        J += [i+1]
        K += [len(cubo)-2]

    I, J, K = I+[23], J+[16], K+[len(cubo)-1]
    for i in range(16, 23):
        I += [i]
        J += [i+1]
        K += [len(cubo)-1]

    xline1, yline1, zline1 = [s[0] for s in sup], [s[1] for s in sup], [s[2] for s in sup]
    xline2, yline2, zline2 = [s[0] for s in med], [s[1] for s in med], [s[2] for s in med]
    xline3, yline3, zline3 = [s[0] for s in inf], [s[1] for s in inf], [s[2] for s in inf]
    xline1, yline1, zline1 = xline1+[xline1[0]], yline1+[yline1[0]], zline1+[zline1[0]]
    xline2, yline2, zline2 = xline2+[xline2[0]], yline2+[yline2[0]], zline2+[zline2[0]]
    xline3, yline3, zline3 = xline3+[xline3[0]], yline3+[yline3[0]], zline3+[zline3[0]]

    xline4, yline4, zline4 = [cubo[0][0], cubo[8][0], cubo[16][0]], [cubo[0][1], cubo[8][1], cubo[16][1]], [cubo[0][2], cubo[8][2], cubo[16][2]]
    xline5, yline5, zline5 = [cubo[2][0], cubo[10][0], cubo[18][0]], [cubo[2][1], cubo[10][1], cubo[18][1]], [cubo[2][2], cubo[10][2], cubo[18][2]]
    xline6, yline6, zline6 = [cubo[4][0], cubo[12][0], cubo[20][0]], [cubo[4][1], cubo[12][1], cubo[20][1]], [cubo[4][2], cubo[12][2], cubo[20][2]]
    xline7, yline7, zline7 = [cubo[6][0], cubo[14][0], cubo[22][0]], [cubo[6][1], cubo[14][1], cubo[22][1]], [cubo[6][2], cubo[14][2], cubo[22][2]]

    xline8 = [cubo[-2][0], cubo[1][0], cubo[9][0], cubo[17][0], cubo[-1][0], cubo[21][0], cubo[13][0], cubo[5][0], cubo[-2][0]]
    yline8 = [cubo[-2][1], cubo[1][1], cubo[9][1], cubo[17][1], cubo[-1][1], cubo[21][1], cubo[13][1], cubo[5][1], cubo[-2][1]]
    zline8 = [cubo[-2][2], cubo[1][2], cubo[9][2], cubo[17][2], cubo[-1][2], cubo[21][2], cubo[13][2], cubo[5][2], cubo[-2][2]]

    xline9 = [cubo[-2][0], cubo[3][0], cubo[11][0], cubo[19][0], cubo[-1][0], cubo[23][0], cubo[15][0], cubo[7][0], cubo[-2][0]]
    yline9 = [cubo[-2][1], cubo[3][1], cubo[11][1], cubo[19][1], cubo[-1][1], cubo[23][1], cubo[15][1], cubo[7][1], cubo[-2][1]]
    zline9 = [cubo[-2][2], cubo[3][2], cubo[11][2], cubo[19][2], cubo[-1][2], cubo[23][2], cubo[15][2], cubo[7][2], cubo[-2][2]]

    fig = go.Figure(data=[
      go.Mesh3d(x=X, y=Y, z=Z, i=I, j=J, k=K, facecolor=colors, lighting=dict(diffuse=0.9), hoverinfo='skip'),

      go.Scatter3d(x=xline1, y=yline1, z=zline1, mode='lines', line=dict(color='black', width=10), hoverinfo='skip'),
      go.Scatter3d(x=xline2, y=yline2, z=zline2, mode='lines', line=dict(color='black', width=10), hoverinfo='skip'),
      go.Scatter3d(x=xline3, y=yline3, z=zline3, mode='lines', line=dict(color='black', width=10), hoverinfo='skip'),
      go.Scatter3d(x=xline4, y=yline4, z=zline4, mode='lines', line=dict(color='black', width=10), hoverinfo='skip'),
      go.Scatter3d(x=xline5, y=yline5, z=zline5, mode='lines', line=dict(color='black', width=10), hoverinfo='skip'),
      go.Scatter3d(x=xline6, y=yline6, z=zline6, mode='lines', line=dict(color='black', width=10), hoverinfo='skip'),
      go.Scatter3d(x=xline7, y=yline7, z=zline7, mode='lines', line=dict(color='black', width=10), hoverinfo='skip'),
      go.Scatter3d(x=xline8, y=yline8, z=zline8, mode='lines', line=dict(color='black', width=10), hoverinfo='skip'),
      go.Scatter3d(x=xline9, y=yline9, z=zline9, mode='lines', line=dict(color='black', width=10), hoverinfo='skip'),
      ],

      layout=go.Layout(
        paper_bgcolor='lightblue',
        title_text="Rubik\'s cube 2x2",
        width=500,
        height=300,
        scene=dict(
          xaxis_visible=False,
          yaxis_visible=False,
          zaxis_visible=False,
          camera=dict(
            up=dict(x=0, y=0, z=1),
            center=dict(x=0, y=0, z=0),
            eye=dict(x=1.5, y=-1.5, z=1.5)
          )
        ),
        margin=dict(l=0, r=0, b=0, t=0),
        showlegend=False,
        updatemenus=[
          dict(
            type="buttons",
            direction="right",
            x=0.57,
            y=1.2,
            pad={"t": 20},
            buttons=list([
              dict(
                label="Start",
                method="animate",
                args=[None, 
                      dict(frame=dict(duration=duration, redraw=True), fromcurrent=True, transition=dict(duration=0)),
                ]                                            
              ),
            ])
          ),    
        ]
      ),
    )

    return fig

### 2D graph

In [8]:
def draw(matrix):
    """
    Drawing colors in a 2d plane
    """
    def grid(x, y, color):
        dicc = {
            0: 'white',
            1: 'yellow',
            2: 'green',
            3: 'orange',
            4: 'blue',
            5: 'red',
            6: 'black'
        }
        plt.fill([x, x+1, x+1, x], [y, y, y+1, y+1], color=dicc[color], edgecolor='black')

    def draw_grid(n, m):
        colors = ['red', 'green']
        fig, ax = plt.subplots()
        for i in range(m):
            for j in range(n):
                grid(i, -j, int(matrix[j, i]))
        plt.show()

    draw_grid(matrix.shape[0], matrix.shape[1])

### Movements

In [9]:
def rotate_submatrix(submatrix, times):
    """
    Rotate a submatrix 90° n-times
    """
    for _ in range(times):
        submatrix = coo_matrix(submatrix)
        rows, cols = submatrix.row, submatrix.col
        data = submatrix.data

        nonzero_rows = [row for row, value in zip(rows, data) if value != 0]
        nonzero_cols = [col for col, value in zip(cols, data) if value != 0]
        nonzero_data = [value for value in data if value != 0]

        new_rows = [submatrix.shape[0] - row - 1 for row in nonzero_cols]
        new_cols = nonzero_rows
        new_data = nonzero_data

        rotated_matrix = coo_matrix((new_data, (new_rows, new_cols)), shape=(submatrix.shape[1], submatrix.shape[0]))
        submatrix = rotated_matrix.toarray()
    return submatrix

def move_cols(matrix, times):
    """
    Translate a submatrix to left n-times
    """
    times = times if times >= 0 else 8 + times
    m = matrix.copy()
    for _ in range(times):
        col_0 = m[:, 0].copy()
        for j in range(1, m.shape[1]):
            m[:, j-1] = m[:, j]
        m[:, -1] = col_0
    return m

def move_rows(matrix, times):
    """
    Translate a submatrix to up n-times
    """
    times = times if times >= 0 else 6 + times
    m = matrix.copy()
    for _ in range(times):
        row_0 = m[0, :].copy()
        for i in range(1, m.shape[0]):
            m[i-1, :] = m[i, :]
        m[-1, :] = row_0
    return m

In [10]:
direction = {0:0, 90: 3, 180: 2, 270: 1, -90: 1, -180: 2, -270: 3}

def accommodate_face(submatrixA, submatrixB, angle):
    """
    Swap positions of submatrix (Destination, origin, angle)
    """
    aux = submatrixA.copy()
    submatrixA = submatrixB
    submatrixB = aux
    submatrixA = rotate_submatrix(submatrixA, direction[angle])
    return submatrixA, submatrixB

def move(matrix, movement):
    """
    Do a rotation (move) of the cube using a representation matrix
    """
    matrix_aux = matrix.copy()

    if movement == 'F':
        matrix_aux[1:5, 1:5] = rotate_submatrix(matrix_aux[1:5, 1:5], direction[90])
    if movement == 'F\'':
        matrix_aux[1:5, 1:5] = rotate_submatrix(matrix_aux[1:5, 1:5], direction[-90])
    if movement == 'F2':
        matrix_aux[1:5, 1:5] = rotate_submatrix(matrix_aux[1:5, 1:5], direction[180])

    if movement == 'R' or movement == 'R\'' or movement == 'R2':
        matrix_aux[0:2, 4:6], matrix_aux[0:2, 2:4] = accommodate_face(matrix_aux[0:2, 4:6], matrix_aux[0:2, 2:4], 90)
        matrix_aux[4:6, 4:6], matrix_aux[4:6, 2:4] = accommodate_face(matrix_aux[4:6, 4:6], matrix_aux[4:6, 2:4], -90)

        if movement == 'R':
            matrix_aux[1:5, 3:7] = rotate_submatrix(matrix_aux[1:5, 3:7], direction[90])
        if movement == 'R\'':
            matrix_aux[1:5, 3:7] = rotate_submatrix(matrix_aux[1:5, 3:7], direction[-90])
        if movement == 'R2':
            matrix_aux[1:5, 3:7] = rotate_submatrix(matrix_aux[1:5, 3:7], direction[180])

        matrix_aux[0:2, 2:4], matrix_aux[0:2, 4:6] = accommodate_face(matrix_aux[0:2, 2:4], matrix_aux[0:2, 4:6], -90)
        matrix_aux[4:6, 2:4], matrix_aux[4:6, 4:6] = accommodate_face(matrix_aux[4:6, 2:4], matrix_aux[4:6, 4:6], 90)

    if movement == 'B' or movement == 'B\'' or movement == 'B2':
        matrix_aux = move_cols(matrix_aux, 2)

        matrix_aux[0:2, 4:6], matrix_aux[0:2, 0:2] = accommodate_face(matrix_aux[0:2, 4:6], matrix_aux[0:2, 0:2], 180)
        matrix_aux[4:6, 4:6], matrix_aux[4:6, 0:2] = accommodate_face(matrix_aux[4:6, 4:6], matrix_aux[4:6, 0:2], -180)

        if movement == 'B':
            matrix_aux[1:5, 3:7] = rotate_submatrix(matrix_aux[1:5, 3:7], direction[90])
        if movement == 'B\'':
            matrix_aux[1:5, 3:7] = rotate_submatrix(matrix_aux[1:5, 3:7], direction[-90])
        if movement == 'B2':
           matrix_aux[1:5, 3:7] = rotate_submatrix(matrix_aux[1:5, 3:7], direction[180])

        matrix_aux[0:2, 0:2], matrix_aux[0:2, 4:6] = accommodate_face(matrix_aux[0:2, 0:2], matrix_aux[0:2, 4:6], -180)
        matrix_aux[4:6, 0:2], matrix_aux[4:6, 4:6] = accommodate_face(matrix_aux[4:6, 0:2], matrix_aux[4:6, 4:6], 180)

        matrix_aux = move_cols(matrix_aux, -2)
    
    if movement == 'L' or movement == 'L\'' or movement == 'L2':
        matrix_aux = move_cols(matrix_aux, -2)

        matrix_aux[0:2, 2:4], matrix_aux[0:2, 4:6] = accommodate_face(matrix_aux[0:2, 2:4], matrix_aux[0:2, 4:6], -90)
        matrix_aux[4:6, 2:4], matrix_aux[4:6, 4:6] = accommodate_face(matrix_aux[4:6, 2:4], matrix_aux[4:6, 4:6], 90)

        if movement == 'L':
            matrix_aux[1:5, 1:5] = rotate_submatrix(matrix_aux[1:5, 1:5], direction[90])
        if movement == 'L\'':
            matrix_aux[1:5, 1:5] = rotate_submatrix(matrix_aux[1:5, 1:5], direction[-90])
        if movement == 'L2':
            matrix_aux[1:5, 1:5] = rotate_submatrix(matrix_aux[1:5, 1:5], direction[180])

        matrix_aux[0:2, 4:6], matrix_aux[0:2, 2:4] = accommodate_face(matrix_aux[0:2, 4:6], matrix_aux[0:2, 2:4], 90)
        matrix_aux[4:6, 4:6], matrix_aux[4:6, 2:4] = accommodate_face(matrix_aux[4:6, 4:6], matrix_aux[4:6, 2:4], -90)

        matrix_aux = move_cols(matrix_aux, 2)
    
    if movement == 'U':
        matrix_aux[2, :] = move_cols(matrix_aux[2, :].reshape(1, len(matrix_aux[2, :])), 2)
        matrix_aux[0:2, 2:4] = rotate_submatrix(matrix_aux[0:2, 2:4], direction[90])
    if movement == 'U\'':
        matrix_aux[2, :] = move_cols(matrix_aux[2, :].reshape(1, len(matrix_aux[2, :])), -2)
        matrix_aux[0:2, 2:4] = rotate_submatrix(matrix_aux[0:2, 2:4], direction[-90])
    if movement == 'U2':
        matrix_aux[2, :] = move_cols(matrix_aux[2, :].reshape(1, len(matrix_aux[2, :])), 4)
        matrix_aux[0:2, 2:4] = rotate_submatrix(matrix_aux[0:2, 2:4], direction[180])

    if movement == 'D':
        matrix_aux[3, :] = move_cols(matrix_aux[3, :].reshape(1, len(matrix_aux[3, :])), -2)
        matrix_aux[4:6, 2:4] = rotate_submatrix(matrix_aux[4:6, 2:4], direction[90])
    if movement == 'D\'':
        matrix_aux[3, :] = move_cols(matrix_aux[3, :].reshape(1, len(matrix_aux[3, :])), 2)
        matrix_aux[4:6, 2:4] = rotate_submatrix(matrix_aux[4:6, 2:4], direction[-90])
    if movement == 'D2':
        matrix_aux[3, :] = move_cols(matrix_aux[3, :].reshape(1, len(matrix_aux[3, :])), -4)
        matrix_aux[4:6, 2:4] = rotate_submatrix(matrix_aux[4:6, 2:4], direction[180])
    return matrix_aux

### Objectives matrix

In [11]:
def all_objectives(objective_matrix):
    """
    Get all posible objective matrix, all posible solutions positions of the cube
    """
    objectives_matrix = [objective_matrix]
    copy_matrix = objective_matrix.copy()

    for _ in range(3):
        copy_matrix = move_cols(copy_matrix, 2)
        copy_matrix[0:2, 2:4], copy_matrix[0:2, 0:2] = accommodate_face(copy_matrix[0:2, 2:4], copy_matrix[0:2, 0:2], 90)
        copy_matrix[4:6, 2:4], copy_matrix[4:6, 0:2] = accommodate_face(copy_matrix[4:6, 2:4], copy_matrix[4:6, 0:2], -90)
        objectives_matrix += [copy_matrix]

    copy_matrix = objective_matrix.copy()
    for i in range(3):
        copy_matrix = move_rows(copy_matrix, -2)
        copy_matrix[2:4, 0:2], copy_matrix[4:6, 0:2] = accommodate_face(copy_matrix[2:4, 0:2], copy_matrix[4:6, 0:2], 90)
        copy_matrix[2:4, 4:6], copy_matrix[4:6, 4:6] = accommodate_face(copy_matrix[2:4, 4:6], copy_matrix[4:6, 4:6], -90)
        copy_matrix[4:6, 6:8], copy_matrix[2:4, 6:8] = accommodate_face(copy_matrix[4:6, 6:8], copy_matrix[2:4, 6:8], 0)
        copy_matrix[0:2, 2:4], copy_matrix[2:4, 6:8] = accommodate_face(copy_matrix[0:2, 2:4], copy_matrix[2:4, 6:8], 0)
        copy_matrix[0:2, 2:4] = rotate_submatrix(copy_matrix[0:2, 2:4], 2)
        copy_matrix[2:4, 6:8] = rotate_submatrix(copy_matrix[2:4, 6:8], 2)
        if i != 1:
            objectives_matrix += [copy_matrix] 
    
    for i in range(6):
        copy_matrix = objectives_matrix[i].copy()
        for _ in range(3):
            copy_matrix[0:6, 0:6] = rotate_submatrix(copy_matrix[0:6, 0:6], 1)
            copy_matrix[2:4, 6:8] = rotate_submatrix(copy_matrix[2:4, 6:8], 3)
            objectives_matrix += [copy_matrix]
            copy_matrix = copy_matrix.copy()

    return objectives_matrix

def choice_objective_matrix(initial_state, objectives_matrix):
    """
    Get the objective matrix that count with less moves to solve the cube
    """
    min_h, best_objective = float('inf'), None
    for objective in objectives_matrix:
        h = heuristic(initial_state, objective)
        if h < min_h:
            min_h, best_objective = h, objective
    return best_objective

### The initial state must be generated by applying random steps to a fixed cube (mix cube)

In [12]:
def mix_cube(matrix_cube, movimientos, duration=1000):
    """
    Mix the cube with random steps and show with 3d graph
    """
    initial_cube = show_cube(matrix_cube, duration)
    figs_data = [initial_cube.data]

    #np.random.seed(2)
    moves = np.random.choice(movements, movimientos)
    last_fig, last_matrix_cube = initial_cube, matrix_cube
    for move_face in moves:
        new_matrix_move = move(last_matrix_cube, move_face)
        fig_aux = copy.deepcopy(last_fig)
        fig_aux.data[0].update(facecolor=get_color_for_cube3D(new_matrix_move))
        figs_data.append(fig_aux.data)
        last_fig, last_matrix_cube = fig_aux, new_matrix_move

    titles = ['Rubik\'s cube 2x2'] + [f'Movements: {names_movements[m]}' for m in moves]
    frames= [go.Frame(layout=dict(title=dict(text=titles[i])), data=fig_data) for i, fig_data in enumerate(figs_data)]
    initial_cube.update(frames=frames)
    initial_cube.show()
    return last_matrix_cube, moves

### Heuristic

In [13]:
def heuristic(state, objective_matrix):
    """
    Count bad position of colors in the cube
    """
    d = 0
    for i in range(state.shape[0]):
        for j in range(state.shape[1]):
            if state[i][j] != objective_matrix[i][j]:
                d += 1
    return d

In [14]:
def heuristic2(node_matrix, objective_matrix):
    """
    Heuristic with Manhattan distance in a cube 3d using a matrix
    """
    points_3d = [(0.5, 1.5, 2), (1.5, 1.5, 2), (0.5, 0.5, 2), (1.5, 0.5, 2),
                (0, 1.5, 1.5), (0, 0.5, 1.5), (0.5, 0, 1.5), (1.5, 0, 1.5), (2, 0.5, 1.5), (2, 1.5, 1.5), (1.5, 2, 1.5), (0.5, 2, 1.5),
                (0, 1.5, 0.5), (0, 0.5, 0.5), (0.5, 0, 0.5), (1.5, 0, 0.5), (2, 0.5, 0.5), (2, 1.5, 0.5), (1.5, 2, 0.5), (0.5, 2, 0.5),
                (0.5, 0.5, 0), (1.5, 0.5, 0), (0.5, 1.5, 0), (1.5, 1.5, 0)]


    values_objective = [objective_matrix[i][j] for i in range(objective_matrix.shape[0]) for j in range(objective_matrix.shape[1]) if objective_matrix[i][j] != 0]
    values_node = [node_matrix[i][j] for i in range(node_matrix.shape[0]) for j in range(node_matrix.shape[1]) if node_matrix[i][j] != 0]

    dicc_node = {}
    for i, p in enumerate(points_3d):
        dicc_node[values_node[i]] = p

    distance = 0
    for i, value in enumerate(values_objective):
        if points_3d[i] != dicc_node[value]:
            distance += abs(dicc_node[value][0] - points_3d[i][0]) + abs(dicc_node[value][1] - points_3d[i][1]) + abs(dicc_node[value][2] - points_3d[i][2])

    return distance

### First attempt A*

In [15]:
def first_a_star(mix_matrix_cube):
    """
    Implementation of A* search for solve cube 2x2
    """
    initial = (mix_matrix_cube.copy(), 'Rubik\'s cube 2x2')
    print('INITIAL STATE \n', initial)

    states = [initial]
    root_states = [initial]
    nodes = []
    root_nodes = []
    distances = []

    objectives = all_objectives(matrix_cube.copy())
    objective_matrix = choice_objective_matrix(initial[0], objectives)

    actual = copy.deepcopy(initial)
    actual_distance = .0

    sw = True
    iterations = 0

    print(color_amarillo, 'A* search...', color_blanco)
    # A* search
    while(sw):
        print('ACTUAL STATE \n', actual)
        iterations += 1

        print(color_amarillo, 'Generatare children...', color_blanco)

        # Generatare children
        for movement in movements:
          node = move(actual[0].copy(), movement)

          if [*states]:
            node = [] if any([np.array_equal(node, s) for s in [*zip(*states)][0]]) else node
          if [*nodes]:
            node = [] if any([np.array_equal(node, n) for n in [*zip(*nodes)][0]]) else node

          if list(node):
            print(f'CHILD (movement {movement})')
            nodes += [(node, movement)]
            distances += [actual_distance + 1.0]
            root_nodes += [actual]

        print(color_amarillo, 'Calculate heuristic...', color_blanco)
        best_index_nodes, best_weighted = 0, float('inf')
        if nodes:
          # Heuristic for each child
          for i, node in enumerate(nodes):
            distance = distances[i]
            heuristic = heuristic2(node[0], objective_matrix)
            weighted = 0.8*heuristic + 0.2*distance
            # weighted = heuristic + distance
            print(f'Child {i} movement {node[1]}', weighted)
            if weighted < best_weighted:
              best_weighted = weighted
              best_index_nodes = i
          
          actual = nodes.pop(best_index_nodes)
          """
          for i, state in enumerate(states):
            if np.array_equal(state, actual):
              root_states.pop(i)
              states.remove(state)
              break
          """
          actual_distance = distances.pop(best_index_nodes)
          root_states += [root_nodes.pop(best_index_nodes)]
          states += [actual]
        else:
          sw = False

        sw = False if np.array_equal(actual[0], objective_matrix) else sw

    # Solution organization
    state = states.pop()
    route = [state]
    root_state = root_states.pop()

    while(not np.array_equal(root_state[0], initial[0])):
      route = [root_state] + route
      conn =  [i for i, s in enumerate(states) if np.array_equal(s[0], root_state[0])][0]
      state = states.pop(conn)
      root_state = root_states.pop(conn)
    route = [initial] + route

    print(color_azul, 'SOLUTION', color_blanco)
    for i, r in enumerate(route):
      print(i+1, r)

    print(color_amarillo, f'Iterations: {iterations}', color_blanco)
    return route

### Second and definitive attempt A*

In [16]:
def successors(actual):
    """
    Generating children of an actual state
    """
    nodes = [[move(actual.copy(), movement), movement] for movement in movements]
    return nodes

def choice_best_score(candidates):
    """
    Get the best state of open list for continue iterations
    """
    min_score, best_index_candidate = float('inf'), None
    for i, (score, state, path) in enumerate(candidates):
        if score < min_score:
            min_score = score
            best_index_candidate = i
    return best_index_candidate

def a_star(initial_state, objectives_matrix):
    """
    Implementation of A* search for solve cube 2x2
    """
    print('INITIAL STATE \n', initial_state)
    file.write('INITIAL STATE: \n' + str(initial_state) + '\n')
    objective_matrix = choice_objective_matrix(initial_state, objectives_matrix)
    # Close list
    explored = []
    # Open list
    candidates = [(heuristic(initial_state, objective_matrix), initial_state, [])]
    iterations = 0

    print(color_amarillo, 'A* Search...', color_blanco)
    file.write('\nA* Search... \n')
    while candidates:
        iterations += 1

        (score, state, path) = candidates.pop(choice_best_score(candidates))
        print('ACTUAL STATE \n', state, [names_movements[p] for p in path])
        file.write('\nACTUAL STATE \n' +  str(state) + '\nactual path: ' + str([names_movements[p] for p in path]) + '\n')

        if np.array_equal(state, objective_matrix):
            return path, iterations

        explored += [state]

        print(color_amarillo, 'Generating Children and Calculating Heuristics...', color_blanco)
        file.write('\nGenerating Children and Calculating Heuristics... \n')
        for i, (successor, movement) in enumerate(successors(state)):
            if not np.any([np.array_equal(successor, e) for e in explored]):
                new_path = path + [movement]
                h = heuristic(successor, objective_matrix)
                new_score = len(new_path) + h
                candidates += [(new_score, successor, new_path)]
                print(f'CHILD {i+1} movement {names_movements[movement]}, d={len(new_path)}, h={h}')
                file.write(f'CHILD {i+1} movement {names_movements[movement]}, d={len(new_path)}, h={h} \n')

    return None

In [17]:
def show_route(initial_matrix, route_movements, duration=1000):
    """
    Show solution in a graph 3d
    """
    route = [initial_matrix]
    for m in route_movements:
        initial_matrix = move(initial_matrix, m)
        route += [initial_matrix]
    initial_cube = show_cube(route[0], duration)
    figs_data = [initial_cube.data]

    last_fig, last_matrix_cube = initial_cube, matrix_cube
    for matrix_move in route[1:]:
        fig_aux = copy.deepcopy(last_fig)
        fig_aux.data[0].update(facecolor=get_color_for_cube3D(matrix_move))
        figs_data.append(fig_aux.data)
        last_fig, last_matrix_cube = fig_aux, matrix_move

    moves = ['Rubik\'s cube 2x2'] + [f'Movements: {names_movements[m]}' for m in route_movements]
    frames = [go.Frame(layout=dict(title=dict(text=moves[i])), data=fig_data) for i, fig_data in enumerate(figs_data)]
    initial_cube.update(frames=frames)
    initial_cube.show(mode='external')

### Main

Open .txt file

In [18]:
if os.path.exists('output.txt'):
    os.remove('output.txt')

In [19]:
file = open("output.txt", "w")
file.write("Welcome to our solution ಥ‿ಥ\n\n");

Mix cube

In [20]:
mix_matrix_cube, moves = mix_cube(matrix_cube, 7)

Show movements of mix cube

In [21]:
file.write('Random steps: ' + str([names_movements[m] for m in moves]) + '\n')
[names_movements[m] for m in moves]

['red-left',
 'orange-left',
 'orange-right',
 'orange-left',
 'green-left',
 'blue-right',
 'red-left']

A* solution

In [22]:
solution, iterations = a_star(mix_matrix_cube.copy(), all_objectives(matrix_cube.copy()))

INITIAL STATE 
 [[0.  0.  6.3 1.4 0.  0.  0.  0. ]
 [0.  0.  3.1 3.3 0.  0.  0.  0. ]
 [5.4 1.3 2.2 2.4 6.1 3.2 4.1 2.3]
 [5.2 6.4 4.4 4.2 1.2 3.4 4.3 2.1]
 [0.  0.  5.3 5.1 0.  0.  0.  0. ]
 [0.  0.  1.1 6.2 0.  0.  0.  0. ]]
 A* Search... 
ACTUAL STATE 
 [[0.  0.  6.3 1.4 0.  0.  0.  0. ]
 [0.  0.  3.1 3.3 0.  0.  0.  0. ]
 [5.4 1.3 2.2 2.4 6.1 3.2 4.1 2.3]
 [5.2 6.4 4.4 4.2 1.2 3.4 4.3 2.1]
 [0.  0.  5.3 5.1 0.  0.  0.  0. ]
 [0.  0.  1.1 6.2 0.  0.  0.  0. ]] []
 Generating Children and Calculating Heuristics... 
CHILD 1 movement orange-right, d=1, h=12
CHILD 2 movement blue-right, d=1, h=24
CHILD 3 movement red-right, d=1, h=24
CHILD 4 movement green-right, d=1, h=18
CHILD 5 movement yellow-right, d=1, h=21
CHILD 6 movement white-right, d=1, h=21
CHILD 7 movement orange-left, d=1, h=18
CHILD 8 movement blue-left, d=1, h=24
CHILD 9 movement red-left, d=1, h=24
CHILD 10 movement green-left, d=1, h=18
CHILD 11 movement yellow-left, d=1, h=21
CHILD 12 movement white-left, d=1, h=21
AC

Show A* solution

In [23]:
show_route(mix_matrix_cube.copy(), solution)

Movements of A* solution

In [24]:
print(color_azul, 'SOLUCIÓN', color_blanco)
file.write('\n SOLUTION: \n' + str([names_movements[s] for s in solution]) + '\n')
print([names_movements[s] for s in solution])
print(color_amarillo, f'Iterations: {iterations}', color_blanco)
file.write(f'Iterations: {iterations}')
file.write('\n \n (✿◠‿◠) Vielen Dank ʕ•́ᴥ•̀ʔっ♡');

 SOLUCIÓN 
['orange-right', 'green-right', 'green-right']
 Iterations: 4 


Close .txt file

In [25]:
file.close() 